# ARIMA
- Produces Crowd Predicitions based on weather and user selected location

### Load the Libaries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from pmdarima import AutoARIMA,auto_arima
import warnings
warnings.filterwarnings("ignore")
import pickle
import os
from datetime import datetime,timedelta,date
import holidays as hl

import openmeteo_requests
import requests_cache
from retry_requests import retry
str_trim_date = pd.to_datetime('2021-01-01').to_datetime64()# The start of the dataset
end_trim_date = pd.to_datetime('2025-12-31').to_datetime64()# The end of the dataset, shouldn't assume both actual end at this date for each location

### Load the data set ensure both the dataframe date range and date are in correct format for ARIMA model

In [3]:
Auck_peds = pd.read_csv("data_weather/Final/Auckland_Pedestrian_daily.csv")
Dub_peds = pd.read_csv("data_weather/Final/Dublin_Pedestrian_daily.csv")

df = pd.concat([Auck_peds, Dub_peds],ignore_index=True)

df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x).to_datetime64())
df = df.sort_values(['Location_ID','Date'])

In [4]:
[df['Date'].values]

[array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
        '2021-01-03T00:00:00.000000000', ...,
        '2025-12-29T00:00:00.000000000', '2025-12-30T00:00:00.000000000',
        '2025-12-31T00:00:00.000000000'], dtype='datetime64[ns]')]

In [5]:
[end_trim_date],[str_trim_date]

([numpy.datetime64('2025-12-31T00:00:00.000000000')],
 [numpy.datetime64('2021-01-01T00:00:00.000000000')])

### Main ARIMA model
- Model creation
- Data splitting
- Fitting model
- Creates pickel files for each location
    - Need seperate pickel files for forecasting each location 
- ARMIA needs to have even spacing between dates
    if gap then a fill in needs to be done for Y values(Dep Var) & Exog(or X Ind Vars)

In [ ]:
models = {}
for loc in df['Location_ID'].unique():
    print(loc)
    sub = df[df['Location_ID'] == loc] 
    sub = sub.drop_duplicates(subset=['Date'], keep='first').sort_values('Date').set_index('Date')# Important for the ARIMA model to function 

    y = sub['PedsSen_Count'].asfreq('D').interpolate()
    x = sub[['Weather_Temperature',
             'Weather_Wind_Gust',
             'Weather_Relative_Humidity',
             'Weather_Precipitation',
             'Is_Holiday']].asfreq('D').interpolate()

    y = y[(y.index <= end_trim_date) & (y.index >= str_trim_date)] # will change depending on new datasets in the future
    x = x[(x.index <= end_trim_date) & (x.index >= str_trim_date)] # will change depending on new datasets in the future 
    print([x.shape,y.shape])

    split = int(len(y) * 0.8) # keep chrno order 

    scaler = StandardScaler()

    y_train = y.iloc[:split]
    y_test  = y.iloc[split:]

    x_train = scaler.fit_transform(x.iloc[:split])
    x_test  = scaler.transform(x.iloc[split:])
    # x_train = x.iloc[:split]
    # x_test  = x.iloc[split:]

    # # ARIMA parameters
    stepwise = auto_arima(y=y_train, # Dep
                      x=x_train, #Inp
                      seasonal=True,
                      m=7, # 7 day pattern
                      trace=False,
                      error_action='ignore',
                      suppress_warnings=True,
                      )
    
    # # SARIMAx model fitting
    model = SARIMAX(endog=y_train, # Dep
                    exog=x_train, # Indep
                    order=stepwise.order, seasonal_order=stepwise.seasonal_order,
                    enforce_stationarity=False, # Variance and trends aren't constant set to false
                    enforce_invertibility=False)
    results = model.fit(disp=False)

    day = 90 # 3 months
    yt = y_test[:day]
    start_date = yt.index[0] # using the date index 
    end_date   = yt.index[-1]
    print(start_date)
    print(end_date)

    y_pred = results.predict(
        start=start_date,
        end=end_date,
        exog=x_test[:day]
    )

    mae  = mean_absolute_error(yt, y_pred)
    rmse = root_mean_squared_error(yt, y_pred)
    r2   = r2_score(yt, y_pred)

    print("MAE :", mae)
    print("RMSE:", rmse)
    print("R²  :", r2)
    models[f'{loc}'] = results

IRDUB_1
[(1826, 5), (1826,)]
2024-12-31 00:00:00
2025-03-30 00:00:00
MAE : 29727.533457775684
RMSE: 33920.8837994051
R²  : -0.9522218701484291
IRDUB_3
[(1826, 5), (1826,)]
2024-12-31 00:00:00
2025-03-30 00:00:00
MAE : 22485.45514220532
RMSE: 25661.872200089394
R²  : -1.1006789972823663
IRDUB_4
[(1826, 5), (1826,)]
2024-12-31 00:00:00
2025-03-30 00:00:00
MAE : 26603.661096488384
RMSE: 30911.153373521764
R²  : -0.6212837079594891
IRDUB_5
[(1826, 5), (1826,)]
2024-12-31 00:00:00
2025-03-30 00:00:00
MAE : 5494.530908943564
RMSE: 6873.9828773365625
R²  : 0.26864920080583177
IRDUB_8
[(1826, 5), (1826,)]
2024-12-31 00:00:00
2025-03-30 00:00:00
MAE : 156512.6983646796
RMSE: 166722.1696191833
R²  : -4.678986933188882
NZAUK_1
[(1826, 5), (1826,)]
2024-12-31 00:00:00
2025-03-30 00:00:00
MAE : 5805.350942499204
RMSE: 7085.3636179820405
R²  : 0.521310522908309
NZAUK_5
[(1826, 5), (1826,)]
2024-12-31 00:00:00
2025-03-30 00:00:00
MAE : 20543.74602744719
RMSE: 22626.185905700986
R²  : -0.0137475704436

In [7]:
models2 = models.copy()

In [8]:
for loc in df['Location_ID'].unique():
    print(loc)
    sub = df[df['Location_ID'] == loc] 
    sub = sub.drop_duplicates(subset=['Date'], keep='first').sort_values('Date').set_index('Date')# Important for the ARIMA model to function 

    y = sub['PedsSen_Count'].asfreq('D').interpolate()
    x = sub[['Weather_Temperature',
             'Weather_Wind_Gust',
             'Weather_Relative_Humidity',
             'Weather_Precipitation',
             'Is_Holiday']].asfreq('D').interpolate()

    y = y[(y.index <= end_trim_date) & (y.index >= str_trim_date)] # will change depending on new datasets in the future
    x = x[(x.index <= end_trim_date) & (x.index >= str_trim_date)] # will change depending on new datasets in the future 

    split = int(len(y) * 0.8) # keep chrno order 

    scaler = StandardScaler()

    x_ft = scaler.fit_transform(x)

    # # ARIMA parameters
    stepwise = auto_arima(y=y, # Dep
                      x=x_ft, #Inp
                      seasonal=True,
                      m=7, # 7 day pattern
                      trace=False,
                      error_action='ignore',
                      suppress_warnings=True,
                      )
    
    # # SARIMAx model fitting
    model = SARIMAX(endog=y, # Dep
                    exog=x_ft, # Indep
                    order=stepwise.order, seasonal_order=stepwise.seasonal_order,
                    enforce_stationarity=False, # Variance and trends aren't constant set to false
                    enforce_invertibility=False)
    results = model.fit(disp=False)

    models2[f'{loc}'] = results

IRDUB_1
IRDUB_3
IRDUB_4
IRDUB_5
IRDUB_8
NZAUK_1
NZAUK_5
NZAUK_6
NZAUK_7


### Save model pickel files

In [ ]:
os.makedirs("arima_models", exist_ok=True) 
for loc,results in models2.items():
    # Save model
    model_path = f"arima_models/{loc}_arima.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(results, f)

#### Use a test case

In [2]:
# Setup the Open-Meteo API client with cache and retry on error # <--- this is from Open Meteo Api Docs
cache_session = requests_cache.CachedSession('.amriacache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [ ]:
def Weather_Requester(lat:float,long:float) -> pd.DataFrame:
    url = "https://archive-api.open-meteo.com/v1/archive" # Histroical Data
    params = {
        "latitude": lat,
        "longitude": long,
        "start_date": '2026-01-01',
        "end_date": (date.today()-timedelta(days=1)).strftime('%Y-%m-%d'),
        "daily": ["temperature_2m_mean", "wind_gusts_10m_mean", "relative_humidity_2m_mean", "precipitation_sum"],
        "timezone": "auto",
    }
    responses = openmeteo.weather_api(url, params=params)
    # Basically getting the data for the beginning of the trim point of Sep 30 2025 of the dataset to 1 day - current day   
    dly = responses[0].Daily()

    T1 = dly.Variables(0).ValuesAsNumpy() # Np array's 
    W1 = dly.Variables(1).ValuesAsNumpy()
    R1 = dly.Variables(2).ValuesAsNumpy()
    P1 = dly.Variables(3).ValuesAsNumpy()

    url = "https://seasonal-api.open-meteo.com/v1/seasonal" # Future Data
    params = {
        "latitude": lat,
        "longitude": long,
        "forecast_days": 180,
        "timezone": "auto",
        "daily": ["temperature_2m_mean", "wind_speed_10m_mean", "relative_humidity_2m_mean", "precipitation_sum"]
    }
    responses = openmeteo.weather_api(url, params=params)
    dly = responses[0].Daily()

    T2 = dly.Variables(0).ValuesAsNumpy()
    W2 = dly.Variables(1).ValuesAsNumpy()
    R2 = dly.Variables(2).ValuesAsNumpy()
    P2 = dly.Variables(3).ValuesAsNumpy()

    T = np.concatenate((T1,T2))
    W = np.concatenate((W1,W2))
    P = np.concatenate((P1,P2))
    R = np.concatenate((R1,R2))
    
    # Build the final indep array, holiday and time will be added later
    vstk = pd.DataFrame(data = np.vstack((T,W,P,R)).T,
                        columns=['Weather_Temperature',
                                 'Weather_Wind_Gust',
                                 'Weather_Relative_Humidity',
                                 'Weather_Precipitation'])

    return vstk

In [4]:
def Holidayer(df:pd.DataFrame,CCode:str) -> pd.DataFrame:
    # Uses country code and each data to find holiday or not
    df['Is_Holiday'] = df['Date'].apply(lambda x: 1 if hl.country_holidays(country=CCode).get(x) != None else 0)
    return df

In [ ]:
loc = "NZAUK_1" # This was a location to be displayed to user
with open(f"arima_models/{loc}_arima.pkl", "rb") as f:
    model = pickle.load(f) # grab right pickel file

d = datetime(2026,4,23).date()# User specfiies a date -- test
w = Weather_Requester(-36.8485,174.7633) # Grab weather from past and for future
w.insert(4,'Is_Holiday',0)# Inserting these columns to match indep input
w.insert(0,'Date',range(len(w))) # Use range to fill in date indexing numbers 
# Add in the date range from trim point 2026-01-01
w['Date'] = w['Date'].apply(lambda x: datetime(2026,1,1).date() + timedelta(days=x))
h = Holidayer(w,'IE') # Add in the holiday data
h = h.set_index('Date').asfreq('D').interpolate(method='linear') # numeric only
# Send to predict next set of days
pred_mean = pd.DataFrame(model.get_forecast(exog=h,steps=len(h)).predicted_mean)
print(pred_mean)

            predicted_mean
2024-12-31   -54882.062426
2025-01-01   -42565.222219
2025-01-02   -31969.317049
2025-01-03   -37592.613082
2025-01-04   -56649.141805
...                    ...
2026-07-31    27936.884445
2026-08-01    19706.811214
2026-08-02    16321.795412
2026-08-03    21481.221231
2026-08-04    24177.361517

[582 rows x 1 columns]


In [6]:
pred_meancp = pred_mean.copy()
idx = pred_meancp.index[:1][0] # deaaling with timestamps
idx

Timestamp('2024-12-31 00:00:00')

In [7]:
f = pd.Timestamp(d) # convert user specified date
f

Timestamp('2026-04-23 00:00:00')

In [8]:
# how to get the forecasted crowd number at a date for a lcoation
[pred_meancp['predicted_mean'].loc[f]] 

[15957.129645256318]

In [9]:
print(*pred_meancp['predicted_mean'].loc[f - timedelta(days=30):f + timedelta(days=10)].items(),sep='\n')

(Timestamp('2026-03-24 00:00:00'), 4761.584269763247)
(Timestamp('2026-03-25 00:00:00'), 6169.702203966124)
(Timestamp('2026-03-26 00:00:00'), 10125.851482163016)
(Timestamp('2026-03-27 00:00:00'), 7675.329893704147)
(Timestamp('2026-03-28 00:00:00'), 2431.9421352268837)
(Timestamp('2026-03-29 00:00:00'), -657.4489785312471)
(Timestamp('2026-03-30 00:00:00'), 6450.841516207292)
(Timestamp('2026-03-31 00:00:00'), 8554.623450077459)
(Timestamp('2026-04-01 00:00:00'), 7375.255735907071)
(Timestamp('2026-04-02 00:00:00'), 10961.666365855323)
(Timestamp('2026-04-03 00:00:00'), 10930.963591254316)
(Timestamp('2026-04-04 00:00:00'), 7457.410590832056)
(Timestamp('2026-04-05 00:00:00'), -1130.8428918880018)
(Timestamp('2026-04-06 00:00:00'), -79.22915839323105)
(Timestamp('2026-04-07 00:00:00'), 3984.8621916948105)
(Timestamp('2026-04-08 00:00:00'), 4264.681326681872)
(Timestamp('2026-04-09 00:00:00'), 11103.390386679675)
(Timestamp('2026-04-10 00:00:00'), 10961.68851710682)
(Timestamp('2026-0